In [1]:
%load_ext autoreload
%autoreload 2


import pandas as pd
import spacy
nlp = spacy.load("fr_core_news_lg")

from os import getcwd, listdir
from os.path import join, dirname, isfile

PATH_REPO = dirname(getcwd())
PATH_UTILS = join(PATH_REPO, 'utils')
PATH_DATA = join(PATH_REPO, 'data')
PATH_CONFIG = join(PATH_REPO, 'config')
PATH_RESULTS = join(PATH_REPO, 'results')

import sys
sys.path.append(PATH_UTILS)


from baseline import load_t_matcher, run_baseline_fr
from metrics import pipeline_report

from sklearn.metrics import precision_recall_fscore_support as score

import numpy as np

pd.set_option('max_colwidth', None)

In [13]:
data = pd.read_csv(join(PATH_DATA, "test_set_fr.csv"), names = ['text', 'label'], engine="python",skiprows=1)
data['label'] = np.where(data['label']=="Gratitude", 1, 0)
data['text'] = data['text'].astype(str)
texts = data['text'].values.tolist()
data

291

In [3]:
docs = list(nlp.pipe(texts))

In [4]:
t_mat = load_t_matcher(PATH_CONFIG, 't_patterns_fr.json', nlp)

In [5]:
predictions = []
patterns = []
sequences = []

for doc in docs:
  appre, pat, seq = run_baseline_fr(nlp, doc, t_mat)
  predictions.append(appre)
  patterns.append(pat)
  sequences.append(seq)


data['prediction'] = np.where(predictions, 1, 0)
data['patterns'] = patterns
data['matching sequences'] = sequences


In [6]:
data.head()

,text,label,prediction,patterns,matching sequences
0,"USER USER USER USER Merci, connaissant la générosité de transmission du savoir et outils de USER j ai peu de doutes 👏😉",1,1,[merci],[Merci]
1,"Si jamais on vous propose une assurance via USER , surtout ne le faites pas. Je ne vous remercie pas USER USER",0,0,[],[]
2,USER ??? Je donne et c’est comme ça qu’on me remercie,0,0,[],[]
3,USER USER USER soutient la réalisation d'un écocide en Gironde. Dommage que USER n'ait pas suivi les conseils de USER et réformé pour la reconnaissance de l'écocide dans notre droit pénal... USER mérite la prison.,0,0,[],[]
4,Pédocriminalité : l’Église va créer une commission de reconnaissance et de réparation Agression sexuelle Violence sexuelle Religions USER,0,0,[],[]


In [7]:
precision, recall, fscore, support = score(data['label'] ,data['prediction'], average='macro')
print("Precision:\t", precision, "\n", "Recall:\t", recall, "\n", "F1:\t", fscore)

Precision:	 0.809375 
 Recall:	 0.8131443298969072 
 F1:	 0.8112010246259533


## Check the false positives:

In [8]:
print(len(data[(data['label']==0) & (data['prediction']==1)]))
data[(data['label']==0) & (data['prediction']==1)][0:68]

36


,text,label,prediction,patterns,matching sequences
28,je vis mon rêve tous les jours c’est incroyable je suis tellement reconnaissante,0,1,[je/nous/on - Ãªtre - super/... - reconnaissant],[je suis tellement reconnaissante]
40,Mais quel soirée incroyable ! Trop d'émotions d'un coup je n'étais pas prête 🤩 Un immense merci à USER et USER pour vos host du turfu !! 🙏😭❤️ Ça m'a fait grave plaisir ☺️ Et une mention spéciale à PLS Man qui a lâché 5000 bits ! 😱 Trop de générosité ❤️,0,1,[merci],[merci]
57,USER USER On te baise (merci mon reuf),0,1,[merci],[merci]
64,USER tro mims merci bcp !!! son iud 713639267,0,1,[merci],[merci]
69,USER USER USER Je la remercie,0,1,[je/nous/on - vous/te/les/le/la - remercier],[Je la remercie]
79,USER Le corbeau pétainiste doit être viré de tous les lieux publics qu'il pollue. Merci.,0,1,[merci],[Merci]
106,USER Bonjour de nouveau! Merci de partager ce grand fou! 😘,0,1,[merci],[Merci]
146,"One piece est tellement incroyable, je remercie ﷻالله d’avoir commencé cette oeuvre aussi profonde et qui n’est pas seulement un groupe de pote qui vont à l’aventure pout aller chercher un trésor, nan c’est beaucoup plus que ça cède juste incroyable quand j’en parle je suis",0,1,[je/nous/on - remercier],[je remercie ﷻالله]
161,Chaque jour qui passe je suis reconnaissante de ne pas être un homme blanc qui porte un sac à dos Decathlon avec son costume Celio,0,1,[je/nous/on - Ãªtre - reconnaissant],[je suis reconnaissante]
171,"🛑 ATTENTION 🛑 Initialement prévues du 22 novembre au 1er décembre, toutes les représentations de la pièce de théâtre ""Le coup de grâce"" sont annulées en raison de cas de Covid-19 détectés au sein de la troupe. Nous vous remercions pour votre compréhension.",0,1,[je/nous/on - vous/te/les/le/la - remercier],[Nous vous remercions]


## Check the false negatives:

In [9]:
print(len(data[(data['label']==1) & (data['prediction']==0)]))
data[(data['label']==1) & (data['prediction']==0)]

33


,text,label,prediction,patterns,matching sequences
6,"Remerciements adressés à USER , USER et USER par tous ceux qui ont pris la parole, du représentant du site Kinyinya II au Conseiller du Gouverneur de USER , en passant par le délégué de l’administrateur de Mutimbuzi et le médecin provincial",1,0,[],[]
21,Je crois que l’ont peut remercier M Barnier candidats LR à la présidentielle pour la qualité des accords sur la pêche avec les anglais! accord bâclé honteux les pêcheurs doivent demander des comptes à BARNIER,1,0,[],[]
41,"En cette belle journée passée au royaume magique, je tiens à remercier mille fois Brocéliande-Nausicaa et Cédric de la Tour de la Terreur et Lionel de Pirates des Caraïbes pour leur accueil, leur gentillesse et leur sens inouï du spectacle 💖 castcompliment USER",1,0,[],[]
46,USER C’est a nous de te remercier c’est toi la meilleure et qui nous font rire tout les jours !! On t’aime fort et on sera toujours la pour toi tu mérite tout le bonheur du monde ily &lt;33,1,0,[],[]
49,(Remerciements éternels à USER qui a trouvé le titre),1,0,[],[]
53,"Nous tenons à remercier USER pour sa contribution. Sans son soutien, nous ne pourrions pas garantir que le Canada dispose d'une stratégie de dépistage et de sensibilisation par et pour la communauté. USER SID21 DepisterCestLaCle VIH dépistage ITSS",1,0,[],[]
72,USER USER USER USER USER USER C’est que j’ai une ligne éditoriale grand public à respecter 🙃 mais j’apprécie ta générosité ;),1,0,[],[]
94,"Vous souvenez-vous quand vous avez rejoint Twitter ? Moi, oui ! MonAnniversaireTwitter Que le temps passe vite. 😭😭 je tiens à vous remercier pour le soutien.",1,0,[],[]
118,"Le général BOURILLON, commandant la région de gendarmerie d’Occitanie, a remis l’insigne ainsi que le diplôme de la réserve citoyenne à Christine DUPATY, Thierry DEBREUX et Didier LAUROUA. Nos sincères remerciements pour leur engagement à nos côtés 🇫🇷 FortsDeNosReservistes",1,0,[],[]
122,"Sincere gratitude to Swicon OC, keynote speakers, Authors, Co-Authors, Presenters, Sponsors &amp; session chairs of Swicon2021: Rupesh Rajhans, L &amp; T Electrical and Automation. USER USER USER USER USER USER USER USER",1,0,[],[]


## Check the True Positives

In [10]:
print(len(data[(data['label']==1) & (data['prediction']==1)]))
data[(data['label']==1) & (data['prediction']==1)]

99


,text,label,prediction,patterns,matching sequences
0,"USER USER USER USER Merci, connaissant la générosité de transmission du savoir et outils de USER j ai peu de doutes 👏😉",1,1,[merci],[Merci]
9,Un grand merci à Mme Garcia (directrice de la USER à Paris) et aux résidents étudiants de cette même Fondation pour avoir organisé une course nocturne dans le cadre de No Finish Line Monaco. monaco générosité NoFinishLine,1,1,"[grand - merci, merci]","[grand merci, merci]"
10,on remercie orelsan pour ce super album,1,1,[je/nous/on - remercier],[on remercie orelsan]
13,"J'exprime ma reconnaissance aux bienfaiteurs des vulnérables au Tchad et je les remercie de tjrs penser à eux. Il serait bien que les autorités tchadiennes S'investissent à produire aussi suffisamment de sorgho, afin d'éviter de toujours attendre des autres. Allahlekou",1,1,[je/nous/on - vous/te/les/le/la - remercier],[je les remercie]
19,USER Salut Jean Pierre 👋 je te remercie et te souhaite également une très bonne journée 😉🔵⚪👊💪💪,1,1,[je/nous/on - vous/te/les/le/la - remercier],[je te remercie]
...,...,...,...,...,...
377,USER Oui oui merci. 😭 Du moment que ça m’évite d’aller en route.,1,1,[merci],[merci]
394,"USER How_to_demystify_gamer_girls.jpeg merci pour l'humour, j'adore rire",1,1,[merci],[merci]
396,"USER USER USER USER USER USER USER USER Merci Monique, à mon tour de vous souhaiter un Shabbat plein de douceur et de sérénité Je vous remercie de tout cœur pour les messages que vous m'envoyez Shabbat Shalom à vous Monique",1,1,"[merci, je/nous/on - vous/te/les/le/la - remercier]","[Merci, Je vous remercie]"
400,"Très ravi d'avoir sillonné les régions de St-Louis et Matam avec l'Equipe USER 🇺🇳et échangé avec les autorités locales que nous remercions,mais surtout d'avoir constaté l'impact de notre action sur les populations, les défis et opportunités d'interventions unisdanslaction",1,1,[je/nous/on - remercier],"[nous remercions,]"


## Check the True Negatives

In [11]:
print(len(data[(data['label']==0) & (data['prediction']==0)]))
data[(data['label']==0) & (data['prediction']==0)]

255


,text,label,prediction,patterns,matching sequences
1,"Si jamais on vous propose une assurance via USER , surtout ne le faites pas. Je ne vous remercie pas USER USER",0,0,[],[]
2,USER ??? Je donne et c’est comme ça qu’on me remercie,0,0,[],[]
3,USER USER USER soutient la réalisation d'un écocide en Gironde. Dommage que USER n'ait pas suivi les conseils de USER et réformé pour la reconnaissance de l'écocide dans notre droit pénal... USER mérite la prison.,0,0,[],[]
4,Pédocriminalité : l’Église va créer une commission de reconnaissance et de réparation Agression sexuelle Violence sexuelle Religions USER,0,0,[],[]
5,Remercie Dieu tu lui dois tout,0,0,[],[]
...,...,...,...,...,...
418,"USER Ne pas mettre Vladdy Jr trop tôt à Cooperstown 😜 Je suis d'accord. La route est encore longue mais ça serait cool d'avoir le père et le fils. En tout cas, s'il vise cette reconnaissance, fallait qu'il se bouge et il a commencé à le faire. Le plus dur sera de durer.",0,0,[],[]
419,"Il dit : « Pas même moi, à moins qu’Allah ne me couvre du voile de Sa miséricorde et de Sa générosité ». [Muslim, riyad as-salihin n°86]",0,0,[],[]
420,"""Soyez reconnaissant pour la vie que vous avez et oubliez votre vain et présomptueux désir d'en avoir une seconde."" Richard Dawkins (Biologiste) Parce que les promesses n'engagent que ceux qui y croient..... Coran Islam arnaque Mohammed",0,0,[],[]
421,"Ne change rien mon Édouard 😅🤣😅…faut remercier God heiin, tu as peut être six vies!!!",0,0,[],[]
